In [ ]:
chain_ARIMA213 = [sample(ARIMA213(s_train[:, C]), NUTS(200, 0.65), 10000) for C in 1:5]

## Fourier Analysis

Finally for a spectral analysis?

### Plotting ARIMA Model Samples for ARIMA(0, 1, 1)

I will now track the \"change of the change ($\Delta^2)$\", using an ARIMA model for the inflation of Treasury securities using the markov chains montecarlo (MCMC) method for simulating  the change of inflation rate changes for upto 10,000 iterations.

In [ ]:
@model ARIMA010(x) = begin
    T = length(x)
    μ ~ Uniform(-10, 10)
    ϕ ~ Uniform(-1, 1)
    for t in 3:T
        val = x[t-1]                  # ARIMA(0,1,0) portion"
        x[t] ~ Normal(val, 1)
    end
end

@model ARIMA110(x) = begin
    T = length(x)
    μ ~ Uniform(-10, 10)
    ϕ ~ Uniform(-1, 1)
    for t in 3:T
        val = μ +                      # Drift term"
              x[t-1] +                 # ARIMA(0,1,0) portion"
              ϕ * (x[t-1] - x[t-2])    # ARIMA(1,0,0) portion"
        x[t] ~ Normal(val, 1)
    end
end

@model ARIMA011(x) = begin
    T = length(x)
    # Set up error vector.
    ϵ = Vector(undef, T)
    x_hat = Vector(undef, T)
    θ ~ Uniform(-5, 5)
    # Treat the first x_hat as a parameter to estimate.
    x_hat[1] ~ Normal(0, 1)
    ϵ[1] = x[1] - x_hat[1]
    for t in 2:T
        # Predicted value for x.
        x_hat[t] = x[t-1] - θ * ϵ[t-1]
        # Calculate observed error.
        ϵ[t] = x[t] - x_hat[t]
        # Observe likelihood.
        x[t] ~ Normal(x_hat[t], 1)
    end
end

Employing a Markov Chain Montecarlo method to test the models I derived using **statespace modeling**.

## Plotting ARIMA Model Samples Using the Markov Chain Montecarlo (MCMC) Method

### Plotting ARIMA Model Samples for ARIMA(1, 1, 1)

Below is a series of plots containing Summary Statistics for $0-10,000$ runs of the **markov chain montecarlo or MCMC** (above) and the quantiles (below). The graphics on the left measures the **stability of inflation data points** for quantiles running from $-\sigma$ to $\sigma$, and the graphics on the right of the $4 \times 4$ grid measures the density (notice how it is standard normal for Summary Statistics of the differenced inflation points), with quantiles having a uniform distribution, as per the model.

### Plotting ARIMA Model Samples for ARIMA(0, 1, 1)

Below is a series of plots containing Summary Statistics for $0-10,000$ runs of the **markov chain montecarlo or MCMC** (above) and the quantiles (below). The graphics on the left measures the **stability of inflation data points** for quantiles running from $-\sigma$ to $\sigma$, and the graphics on the right of the $4 \times 4$ grid measures the density (notice how it is standard normal for Summary Statistics of the differenced inflation points), with quantiles having a uniform distribution, as per the model.

### Plotting ARIMA Model Samples for ARIMA(0, 1, 3)

Below is a series of plots containing Summary Statistics for $0-10,000$ runs of the **markov chain montecarlo or MCMC** (above) and the quantiles (below). The graphics on the left measures the **stability of inflation data points** for quantiles running from $-\sigma$ to $\sigma$, and the graphics on the right of the $4 \times 4$ grid measures the density (notice how it is standard normal for Summary Statistics of the differenced inflation points), with quantiles having a uniform distribution, as per the model.

### Plotting ARIMA Model Samples for ARIMA(3, 1, 1)

Below is a series of plots containing Summary Statistics for $0-10,000$ runs of the **markov chain montecarlo or MCMC** (above) and the quantiles (below). The graphics on the left measures the **stability of inflation data points** for quantiles running from $-\sigma$ to $\sigma$, and the graphics on the right of the $4 \times 4$ grid measures the density (notice how it is standard normal for Summary Statistics of the differenced inflation points), with quantiles having a uniform distribution, as per the model.

### Plotting ARIMA Model Samples for ARIMA(2, 1, 3)

Below is a plot of Summary Statistics for $0-10,000$ runs of the **markov chain montecarlo or MCMC** (above) and the quantiles (below). The graphics on the left measures the **stability of inflation data points** for quantiles running from $-\sigma$ to $\sigma$, and the graphics on the right of the $4 \times 4$ grid measures the density (notice how it is standard normal for Summary Statistics of the differenced inflation points), with quantiles having a uniform distribution, as per the model.